In [ ]:
# ---------------------------------------------------------
# Script: JSON to CSV Conversion Examples
# Description:
#   Demonstrates multiple methods for converting JSON files
#   (simple, multi-level, nested) into CSV format.
#   Outputs all CSV files into destination/json/.
# ---------------------------------------------------------

In [6]:
import pandas as pd
import os
import json

1. Simple JSON Structure

In [ ]:
simple_json_file_path = "simple_structure.json"

# Load simple JSON file into DataFrame
df_json = pd.read_json(simple_json_file_path)
print(df_json)

# Export to CSV 
output_folder = "destination/json"
os.makedirs(output_folder, exist_ok=True)

csv_file = 'simple_json.csv'
simple_output_path = os.path.join(output_folder, csv_file)
df_json.to_csv(simple_output_path, index=False)

print(f"JSON converted and saved to: {simple_output_path}")

   id    name  math  physics  chemistry
0  A1     Jim    60       66         61
1  A2  Dwight    89       76         51
2  A3   Kevin    79       90         78
JSON converted and saved to: destination/json\simple_json.csv


2. Multi-Level JSON 

In [ ]:
multiple_level_json_file_path = "multiple_level.json"

# Load multi-level JSON into dictionary
with open(multiple_level_json_file_path, "r") as f:
    data = json.load(f)

# Normalize nested JSON (students list inside dictionary)
# 'meta' keeps parent-level fields
df_students = pd.json_normalize(data,
                                record_path="students", 
                                meta=["school_name", "class"])                                                         

print(df_students)

# Export to CSV 
students_output_path = os.path.join(output_folder, "students.csv")

df_students.to_csv(students_output_path, index=False)

print(f"CSV exported successfully to: {students_output_path}")

   id    name  math  physics  chemistry    school_name   class
0  A1     Jim    60       66         61  Dunder Miflin  Year 1
1  A2  Dwight    89       76         51  Dunder Miflin  Year 1
2  A3   Kevin    79       90         78  Dunder Miflin  Year 1
CSV exported successfully to: destination/json\students.csv


3. Nested JSON with Multi Meta Paths

In [ ]:
nested_level_json_file_path = "nested_level.json"

# Load nested JSON into dictionary 
with open(nested_level_json_file_path, "r") as f:
    data = json.load(f)

# Normalize nested JSON with nested meta fields:
# meta fields like ["class", "level"] reference nested keys
df_nested = pd.json_normalize(data, record_path="students", sep="_",
                       meta=["school_name", 
                             ["class","level"], 
                             ["class","room"], 
                             ["class","teacher"]])
print(df_nested)

# Export to CSV
nested_output_path = os.path.join(output_folder, "nested_students.csv")
df_nested.to_csv(nested_output_path, index=False)

print(f"CSV exported successfully to: {nested_output_path}")

   id    name  grade_math  grade_physics  grade_chemistry    school_name  \
0  A1     Jim          60             66               61  Dunder Miflin   
1  A2  Dwight          89             76               51  Dunder Miflin   
2  A3   Kevin          79             90               78  Dunder Miflin   

  class_level class_room class_teacher  
0          12      K1.01          John  
1          12      K1.01          John  
2          12      K1.01          John  
CSV exported successfully to: destination/json\nested_students.csv


In [11]:
# Approach 2: Read seperately and cross join

# Load nested JSON into dictionary 
with open(nested_level_json_file_path, "r") as f:
    data = json.load(f)

# Extract student list as DataFrame
df_students = pd.json_normalize(data["students"], sep="_")

# Extract class/school metadata as separate DataFrame
class_info = {
    "school_name": data["school_name"],
    "class_level": data["class"]["level"],
    "class_room": data["class"]["room"],
    "class_teacher": data["class"]["teacher"]
}

df_class = pd.DataFrame([class_info])

# CROSS JOIN students × class_info
df_final = df_students.merge(df_class, how="cross")

print("\nFinal merged dataset:")
print(df_final)


Final merged dataset:
   id    name  grade_math  grade_physics  grade_chemistry    school_name  \
0  A1     Jim          60             66               61  Dunder Miflin   
1  A2  Dwight          89             76               51  Dunder Miflin   
2  A3   Kevin          79             90               78  Dunder Miflin   

   class_level class_room class_teacher  
0           12      K1.01          John  
1           12      K1.01          John  
2           12      K1.01          John  
